In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
file_name = "df_count_2_part2.csv"   

In [3]:
 
df = pd.read_csv(file_name , parse_dates=["date"])


C:\Users\oussa\AppData\Local\Temp\ipykernel_22044\1571972478.py:1: DtypeWarning: Columns (0,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name , parse_dates=["date"])


In [4]:
df.shape

(8809865, 10)

In [5]:
# Read the country reference file
df_countries = pd.read_csv("CT_COUNTRIES.csv", sep=";", header=None, names=["code", "lang", "name"])

country_dict = df_countries.set_index("code")["name"].to_dict()

In [6]:
# Define columns to ignore when checking duplicates
cols_ignore = ["RECPTCL_FID", "duration_to_next_step", "total_duration", "id"]
cols_check  = [c for c in df.columns if c not in cols_ignore]

# 1. Identify all duplicate rows based on cols_check
dup_mask = df.duplicated(subset=cols_check, keep=False)

# 2. Number of such duplicate rows
num_duplicates = dup_mask.sum()
print(f"Number of duplicate rows (ignoring specified columns): {num_duplicates}")




Number of duplicate rows (ignoring specified columns): 12564


In [7]:
df = df.drop_duplicates(subset=cols_check, keep="first").reset_index(drop=True)

In [8]:
import pandas as pd

ID_COL    = "MAILITM_FID"
EVENT_COL = "EVENT_TYPE_NM"

# 1) pick the first row per parcel (rows are already in correct order)
first_events = (
    df.groupby(ID_COL, sort=False)[EVENT_COL]
      .first()                 # a Series: index = parcel ID, value = first event
)

# 2) count how many parcels have each first event
event_counts = (
    first_events.value_counts()   # Series: index = event, value = count
               .sort_values(ascending=False)
)

# 3) print the result
print("First-event frequencies:")
for event, count in event_counts.items():
    print(f"{event:40} {count:,}")


First-event frequencies:
Réception d'envoi du client (Srt)        484,519
Expédition d'envoi à l'étranger (EDI-reçu) 308,347
Insérer envoi dans sac (Srt)             32,990
Recevoir envoi au bureau d'échange (Ent) 1,251
Expédier envoi à adresse nationale (Ent) 807
Recevoir envoi au bureau de livraison (Ent) 298
Expédier envoi à adresse nationale (Srt) 156
Vaine tentative de livraison d'envoi (Ent) 40
Transmettre envoi à l'agent de livraison (Ent) 11
Livraison d'envoi (Ent)                  9
Recevoir envoi au bureau d'échange (Srt) 1


In [9]:
missing_values = df.isnull().sum()
missing_columns = missing_values[missing_values > 0]
print(missing_columns)

RECPTCL_FID                  3317593
établissement_postal          722332
next_établissement_postal    5720787
duration_to_next_step        1655106
dtype: int64


In [10]:
df = df.drop(columns=["RECPTCL_FID", "duration_to_next_step", "total_duration"])

In [11]:
missing_values = df.isnull().sum()
missing_columns = missing_values[missing_values > 0]
print(missing_columns)

établissement_postal          722332
next_établissement_postal    5720787
dtype: int64


In [12]:

# Assuming 'df' is your DataFrame!

# Identify the rows with missing values
missing_rows = df[df.isnull().any(axis=1)]

# Print the total number of rows that have missing values
print("Total rows with missing values:", missing_rows.shape[0])




Total rows with missing values: 5819132


In [13]:
# Analyze the missing patterns across rows:
# Create a binary pattern for each row where 1 indicates a missing value and 0 indicates non-missing.
missing_pattern = missing_rows.isnull().astype(int)
# Convert each pattern to a tuple to use as a key for groupby
missing_pattern_tuples = missing_pattern.apply(tuple, axis=1)
# Group by the missing pattern and count the number of rows for each pattern
pattern_summary = missing_rows.groupby(missing_pattern_tuples).size().sort_values(ascending=False)

print("\nSummary of missing patterns (tuple of 0's and 1's corresponding to missing values in each column):")
print(pattern_summary)




Summary of missing patterns (tuple of 0's and 1's corresponding to missing values in each column):
(0, 0, 0, 0, 0, 1, 0)    5096800
(0, 0, 0, 1, 0, 1, 0)     623987
(0, 0, 0, 1, 0, 0, 0)      98345
dtype: int64


In [14]:
distinct_values_count = df.nunique()

distinct_values_count = distinct_values_count.sort_values(ascending=True)

print(distinct_values_count)

EVENT_TYPE_NM                     23
EVENT_TYPE_CD                     23
id                               102
next_établissement_postal       3546
établissement_postal            3698
MAILITM_FID                   828429
date                         7002024
dtype: int64


In [15]:

# Count the occurrences of each protocol
event_type = df['EVENT_TYPE_NM'].value_counts()

print(event_type)


EVENT_TYPE_NM
Recevoir envoi au bureau de livraison (Ent)                    1742652
Expédier envoi à adresse nationale (Srt)                       1219055
Expédier envoi à adresse nationale (Ent)                       1190353
Recevoir envoi au bureau d'échange (Ent)                        871789
Vaine tentative de livraison d'envoi (Ent)                      811097
Livraison d'envoi (Ent)                                         714376
Expédition d'envoi à l'étranger (EDI-reçu)                      632476
Insérer envoi dans sac (Srt)                                    567457
Réception d'envoi du client (Srt)                               505053
Transmettre envoi à l'agent de livraison (Ent)                  497818
Recevoir envoi au bureau d'échange (Srt)                         38144
Enregistrer détails d'envoi au bureau d'échange (Srt)             7486
Expédier envoi à la douane (Ent)                                  3984
Renvoyer envoi de la douane (Ent)                              

In [16]:

# Count the occurrences of each protocol
proto_counts = df['établissement_postal'].value_counts()

print(proto_counts)


établissement_postal
ALGER COLIS POSTAUX        1074154
ALGER GARE                  936123
ÉMIRATS ARABES UNIS         639187
ANNABA EL MARSA             211689
CONSTANTINE GARE            175039
                            ...   
BENI-OUARSOUS                    1
EL-MILIA                         1
MEDEA YAHIA FARES                1
GHARDAIA-COLONNEL-LOTFI          1
NAAMA-RP                         1
Name: count, Length: 3698, dtype: int64


In [17]:
# Count the occurrences of each protocol
proto_counts = df['next_établissement_postal'].value_counts()

print(proto_counts)



next_établissement_postal
ALGÉRIE                      632476
ALGER GARE                   325462
CONSTANTINE COLIS POSTAUX    139537
ORAN COLIS POSTAUX           124054
ANNABA EL MARSA               87285
                              ...  
GRAREM-05-JUILLET                 1
CHAHID-CHBANA-SAID                1
OUM-EL-BOUAGHI-TERRE              1
BALLOUL-GARE-ROUTIERE             1
SIRAT                             1
Name: count, Length: 3546, dtype: int64


In [18]:

# Count the occurrences of each protocol
proto_counts = df['id'].value_counts()

print(proto_counts)

s = proto_counts


id
AE    3477076
CZ    3258833
CN     954381
FR     471834
DE     167010
       ...   
UY          7
CR          6
NZ          6
KZ          5
AZ          2
Name: count, Length: 102, dtype: int64


In [19]:
# Map the Series index using the country_dict
s.index = s.index.map(lambda code: country_dict.get(code, code))  # Keep unmapped codes as-is
print(s)

id
ÉMIRATS ARABES UNIS    3477076
TCHÈQUE, RÉPUBLIQUE    3258833
CHINE                   954381
FRANCE                  471834
ALLEMAGNE               167010
                        ...   
URUGUAY                      7
COSTA RICA                   6
NOUVELLE-ZÉLANDE             6
KAZAKHSTAN                   5
AZERBAÏDJAN                  2
Name: count, Length: 102, dtype: int64


In [20]:
print("Min date:", df["date"].min())
print("Max date:", df["date"].max())

Min date: 2020-07-20 07:41:23.217
Max date: 2025-05-21 16:00:58.757


In [21]:
df["date"] = df["date"].str.replace(r'\.\d+$', '', regex=True)
df["date"] = pd.to_datetime(df["date"], infer_datetime_format=True)

C:\Users\oussa\AppData\Local\Temp\ipykernel_22044\579475394.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["date"] = pd.to_datetime(df["date"], infer_datetime_format=True)


In [22]:
df = df.sort_values(by=["MAILITM_FID", "date"])

# Calculate time to next step
df["duration_to_next_step"] = df.groupby("MAILITM_FID")["date"].shift(-1) - df["date"]

# Calculate total duration per ID
first_date = df.groupby("MAILITM_FID")["date"].transform("first")
last_date = df.groupby("MAILITM_FID")["date"].transform("last")
df["total_duration"] = last_date - first_date




In [23]:
# Preview the updated dataset
df[["MAILITM_FID", "date", "EVENT_TYPE_NM", "duration_to_next_step", "total_duration"]].head(10)

,MAILITM_FID,date,EVENT_TYPE_NM,duration_to_next_step,total_duration
1661229,CB167774583US,2024-09-20 05:15:00,Expédition d'envoi à l'étranger (EDI-reçu),6 days 03:00:00,12 days 05:09:46
1661230,CB167774583US,2024-09-26 08:15:00,Expédition d'envoi à l'étranger (EDI-reçu),6 days 00:10:12,12 days 05:09:46
8803225,CB167774583US,2024-10-02 08:25:12,Recevoir envoi au bureau d'échange (Ent),0 days 01:59:34,12 days 05:09:46
8803226,CB167774583US,2024-10-02 10:24:46,Expédier envoi à la douane (Ent),NaT,12 days 05:09:46
1661231,CP068462845TH,2024-10-10 23:05:00,Expédition d'envoi à l'étranger (EDI-reçu),3 days 09:46:44,3 days 09:46:44
8803227,CP068462845TH,2024-10-14 08:51:44,Recevoir envoi au bureau d'échange (Ent),NaT,3 days 09:46:44
1661232,CP070826787TH,2024-06-15 23:05:00,Expédition d'envoi à l'étranger (EDI-reçu),3 days 10:03:26,3 days 10:03:26
8803228,CP070826787TH,2024-06-19 09:08:26,Recevoir envoi au bureau d'échange (Ent),NaT,3 days 10:03:26
8803229,CP080036457Th,2024-04-07 09:52:20,Recevoir envoi au bureau d'échange (Ent),0 days 00:25:29,6 days 00:57:59
8803230,CP080036457Th,2024-04-07 10:17:49,Mettre à jour envoi (Ent),0 days 00:25:38,6 days 00:57:59


In [24]:
# Filter rows where total_duration is NOT zero
zero_duration_df = df[df["total_duration"] == pd.Timedelta(days = 0)]

# Display the first few rows as a check
zero_duration_df.shape


(0, 9)

In [38]:

# Count the occurrences of each protocol
countries = zero_duration_df['id'].value_counts()

print(countries)


Series([], Name: count, dtype: int64)


In [39]:
# Map the Series index using the country_dict
countries.index = countries.index.map(lambda code: country_dict.get(code, code))  
print(countries)

Series([], Name: count, dtype: int64)


In [40]:
# Step 1: Count how many times each ID appears
id_counts = zero_duration_df["MAILITM_FID"].value_counts()

# Step 2: Count how many IDs have a specific count
count_of_counts = id_counts.value_counts().sort_index()

print(count_of_counts)


Series([], Name: count, dtype: int64)


In [41]:
ids_with_0 = id_counts[id_counts == 2].index
df_with_0 = df[df["MAILITM_FID"].isin(ids_with_0)]
df_with_0.head()

,MAILITM_FID,EVENT_TYPE_NM,date,établissement_postal,EVENT_TYPE_CD,next_établissement_postal,id,duration_to_next_step,total_duration


In [42]:

# Count the occurrences of each protocol
event_type = df_with_0['EVENT_TYPE_NM'].value_counts()

print(event_type)



Series([], Name: count, dtype: int64)


In [25]:
# Drop rows where total_duration is zero
df = df[df["total_duration"] != pd.Timedelta(0)]

# Check the shape after dropping
print(df.shape)


(8803577, 9)


In [26]:
df.to_csv(file_name, index=False, encoding='utf-8-sig')